In [1]:
import numpy as np
import pandas as pd
from pandas.io import sql
from sqlalchemy import create_engine
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
%load_ext dotenv
%dotenv ../.env

In [3]:
cnx = create_engine(os.getenv('EDMDB'))

Are there any invalid proxcode values on PLUTO? There are some NULL values.

In [4]:
proxcode_counts = pd.read_sql_query('''SELECT "ProxCode", COUNT(*)
FROM dcp.pluto202
GROUP BY "ProxCode";''', cnx)
proxcode_counts

,ProxCode,count
0,0,141749
1,1,313774
2,2,200383
3,3,200940
4,None,375


#### ProxCode = '2' or '3' (Attached or Semi-Attached Buildings)

In [5]:
sql.execute('''DROP TABLE IF EXISTS dcp.buildings_23;
DROP TABLE IF EXISTS dcp.lot23_no_neighboring_bldg;
DROP TABLE IF EXISTS dcp.lot23_no_neighboring_bldg_1ft;
DROP TABLE IF EXISTS dcp.buildings_1;
DROP TABLE IF EXISTS dcp.lot1_bldg;
DROP TABLE IF EXISTS dcp.lot1_neighboring_bldg;''', cnx)

Get buildings and geometries for lots marked as attached or semi-attached. Limit analysis to those lots having only one building.

In [6]:
sql.execute('''CREATE TABLE dcp.buildings_23 AS
    SELECT CAST(p."BBL" AS TEXT) AS bbl, bin, f.geom as building_geom, p.geom as lot_geom
    FROM dcp.bldg_footprints f, dcp.pluto202 p
    WHERE f.mpluto_bbl = CAST(p."BBL" AS TEXT)
    AND "ProxCode" IN ('2', '3')
    AND "NumBldgs" = 1;
CREATE INDEX lot_geom_idx
  ON dcp.buildings_23
  USING GIST (lot_geom);
CREATE INDEX building_geom_idx
  ON dcp.buildings_23
  USING GIST (building_geom);
''', cnx)
buildings_23 = pd.read_sql_query('''SELECT * FROM dcp.buildings_23;''', cnx)
buildings_23.shape[0]

331926

Get building footprints for lots whose ProxCode is either 'attached' or 'semi-attached', yet have no buildings abutting them. Note that ST_Intersects seemed to work better here than ST_Touches; I'm not sure why. Put results in tables so you have the option of importing them to shapefiles.

In [7]:
sql.execute('''CREATE TABLE dcp.lot23_no_neighboring_bldg AS
SELECT b1.bbl, b1.bin, b1.building_geom, b1.lot_geom, COALESCE(x.count,0) AS count
FROM dcp.buildings_23 b1
LEFT JOIN (
	SELECT b1.bbl as bbl, COUNT(*) AS count
	FROM dcp.bldg_footprints b2, dcp.buildings_23 b1
	WHERE ST_Intersects(b2.geom, b1.building_geom)
	AND b2.mpluto_bbl <> b1.bbl
	AND b2.bin <> b1.bin
	GROUP BY b1.bbl
) x
ON b1.bbl = x.bbl
WHERE COALESCE(x.count,0) = 0;''', cnx)

5477 of 331926 buildings have no abutting building even if they are characterized as attached or semi attached. This is about 1.6%.

In [8]:
lot23_no_neighboring_bldg = pd.read_sql_query('''SELECT * FROM dcp.lot23_no_neighboring_bldg;''', cnx)
lot23_no_neighboring_bldg.shape[0]

5477

Now get only those buildings that don't have another building within one foot.

In [9]:
sql.execute('''CREATE TABLE dcp.lot23_no_neighboring_bldg_1ft AS
SELECT l.*
FROM dcp.lot23_no_neighboring_bldg l
WHERE l.bbl NOT IN (
SELECT a.bbl
    FROM dcp.lot23_no_neighboring_bldg a, dcp.bldg_footprints b
    WHERE ST_DWithin(a.building_geom, b.geom, 1)
    AND a.bin <> b.bin
    AND ST_Distance(a.building_geom, b.geom) > 0)
''', cnx)
lot23_no_neighboring_bldg_1ft = pd.read_sql_query('''SELECT * FROM dcp.lot23_no_neighboring_bldg_1ft;''', cnx)
lot23_no_neighboring_bldg_1ft.shape[0]

4169

#### ProxCode = 1 (Non-Attached Buildings)

Create a table holding only those lots/buildings where the ProxCode = 1 (non-attached buildings).

In [10]:
sql.execute('''CREATE TABLE dcp.buildings_1 AS
SELECT CAST("BBL" AS TEXT) AS bbl, 
b.bin, 
b.geom as building_geom, 
p.geom as lot_geom
FROM dcp.pluto202 p, dcp.bldg_footprints b
WHERE "ProxCode" = '1'
AND "NumBldgs" = 1
AND CAST(p."BBL" AS TEXT) = b.mpluto_bbl;
CREATE INDEX lot_geom_idx_1
  ON dcp.buildings_1
  USING GIST (lot_geom);
CREATE INDEX building_geom_idx_1
  ON dcp.buildings_1
  USING GIST (building_geom);''', cnx)
buildings_1 = pd.read_sql_query('''SELECT * FROM dcp.buildings_1;''', cnx)
buildings_1.shape[0]

162869

Get geometry for lot where ProxCode equals 1 but the building has an abutting building

In [11]:
sql.execute('''CREATE TABLE dcp.lot1_bldg AS
SELECT b1.bbl, b1.bin AS bldg1_bin, b1.building_geom, b2.bin AS footprints_bin
FROM dcp.buildings_1 b1, dcp.bldg_footprints b2
WHERE (b1.bin <> b2.bin)
AND ST_DWithin(b1.building_geom, b2.geom, 0)
AND NOT ST_Equals(b1.building_geom, b2.geom);''', cnx)
lot1_bldg = pd.read_sql_query('''SELECT * FROM dcp.lot1_bldg;''', cnx)
lot1_bldg.shape[0]

13627

Importing the geometry from the buildings characterized as standalone along with neighboring buildings shows that some actually abut other buildings (13627 of 162869 or 8.36%). Put the neighboring buildings in a table so you can import them into shapefiles and view them in ArcMap.

In [12]:
sql.execute('''CREATE TABLE dcp.lot1_neighboring_bldg AS
SELECT b2.mpluto_bbl, b2.bin, b2.geom
FROM dcp.buildings_1 b1, dcp.bldg_footprints b2
WHERE (b1.bin <> b2.bin)
AND ST_DWithin(b1.building_geom, b2.geom, 0)
AND NOT ST_Equals(b1.building_geom, b2.geom);''', cnx)
lot1_neighboring_bldg = pd.read_sql_query('''SELECT * FROM dcp.lot1_neighboring_bldg;''', cnx)
lot1_neighboring_bldg.shape[0]

13627